In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from fastai.learner import *
from fastai.column_data import *

/home/ubuntu/anaconda3/envs/python3/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/envs/python3/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))


In [2]:
path='data/ml-latest-small/'

In [3]:
ratings=pd.read_csv(path+'ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [4]:
movies=pd.read_csv(path+'movies.csv')


In [5]:
val_idxs = get_cv_idxs(len(ratings)); #fastai function get get validation data ?? get_cv_idxs 
wd=2e-4 
n_factors = 50 # Embedding size

In [6]:
cf = CollabFilterDataset.from_csv(path, 'ratings.csv', 'userId', 'movieId', 'rating')
learn = cf.get_learner(n_factors, val_idxs, 64, opt_fn=optim.Adam)

In [ ]:
learn.fit(1e-2, 2, wds=wd,cycle_len=1, cycle_mult=2 ) 

In [ ]:
math.sqrt(0.763) # val_loss is mean sqaure error, so need to take the sqaure root

In [ ]:
preds = learn.predict()

In [ ]:
preds

In [ ]:
len(preds)

In [ ]:
y = learn.data.val_y
sns.jointplot(preds,y,kind='hex',stat_func=None)

In [7]:
# Create  user id that are contiouns, eg: 1,2,3,4,5 not 34,56,78,79
u_uniq = ratings.userId.unique() 
user2idx = {o:i for i,o in enumerate(u_uniq)} 
ratings.userId = ratings.userId.apply(lambda x: user2idx[x])

# Same for movieid

m_uniq = ratings.movieId.unique()
movie2idx = {o:i for i,o in enumerate(m_uniq)}
ratings.movieId = ratings.movieId.apply(lambda x: movie2idx[x])

# the number of users and movies
n_users=int(ratings.userId.nunique()) # 671
n_movies=int(ratings.movieId.nunique()) # 9066

In [8]:
n_movies

9066

In [13]:
# Create Embeddings
class EmbeddingDot(nn.Module):
    def _init_(self, n_user ,n_movies):
        super().__init__()
        # creating our embedding matrix  of size (n_users X n_factors)
        self.u = nn.Embedding(n_users, n_factors) 
        self.m = nn.Embedding(n_movies,n_factors)
        # randomly initalising  elements in embeddings
        self.u.weight.data.uniform_(0,0.05)
        self.m.weight.data.uniform_(0,0.05)
        
    def forward(self, cats, conts):
        users,movies = cats[:,0],cats[:,1]
        u,m = self.u(users),self.m(movies)
        return (u*m).sum(1) # returns the dot product of 
        

In [14]:
x = ratings.drop(['rating', 'timestamp'], axis=1) # just user and movie id
y = ratings['rating'].astype(np.float32) # just the rating id converted to a float

In [15]:
# Put data in a form our model can read
data = ColumnarModelData.from_data_frame(path, val_idxs, x , y, ['userId','movieId'], 64)

In [16]:
wd=1e-5 #weight decay
model = EmbeddingDot(n_users, n_movies).cuda() #instantiate model with no. of user & movies .cuda() to use GPU
opt = optim.SGD(model.parameters(), 1e-1, weight_decay=wd, momentum=0.9)

TypeError: __init__() takes 1 positional argument but 3 were given

In [ ]:
fit(model, data, 3, opt , F.mse_loss)


In [ ]:
set_lrs(opt, 0.01)

In [ ]:
fit(model, data, 5, opt , F.mse_loss)

In [ ]:
model

In [ ]:
model.u

### Add Bias

In [17]:
min_rating,max_rating = ratings.rating.min(), ratings.rating.max()
min_rating,max_rating

(0.5, 5.0)

In [18]:
def get_emb(ni,nf):
    e = nn.Embedding(ni, nf)
    e.weight.data.uniform_(-0.01,0.01)
    return e

class EmbeddingDotBias(nn.Module):
    def __init__(self, n_users, n_movies):
        super().__init__()
        (self.u, self.m, self.ub, self.mb) = [get_emb(*o) for o in [
            (n_users, n_factors), (n_movies, n_factors), (n_users,1), (n_movies,1)
        ]] # run get embeddings matrix for each thing
        
        
    def forward(self, cats, conts):
        users,movies = cats[:,0],cats[:,1]
        um = (self.u(users)* self.m(movies)).sum(1)
        res = um + self.ub(users).squeeze() + self.mb(movies).squeeze()
        res = F.sigmoid(res) * (max_rating-min_rating) + min_rating
        return res
        
        
#     def forward(self,cats,cons):
#         users,movies = cats[:,0],cats[:,1]
#         um = (self.u(users)*self.m(movies).sum(1)) #dot product of user and movie
#         res = um + self.ub(users).squeeze() + self.mb(movies).squeeze() #add on bias
#         res = F.sigmoid(res)*(max_rating-min_rating) + min_rating #make predictions within ratings eg (1-5)
#         return res
        

In [19]:
wd=2e-4
model = EmbeddingDotBias(cf.n_users,cf.n_items).cuda()
opt = optim.SGD(model.parameters(), 1e-1, weight_decay=wd, momentum=0.9)

In [20]:
fit(model,data,3,opt, F.mse_loss)

HBox(children=(IntProgress(value=0, description='Epoch', max=3), HTML(value='')))

epoch      trn_loss   val_loss                                  
    0      0.861542   0.837933  
    1      0.780955   0.817729                                  
    2      0.782289   0.809939                                  



[0.80993885]

In [21]:
math.sqrt(0.8099)

0.8999444427296609

In [22]:
set_lrs(opt, 1e-2)

In [24]:
fit(model,data,3,opt, F.mse_loss)

HBox(children=(IntProgress(value=0, description='Epoch', max=3), HTML(value='')))

epoch      trn_loss   val_loss                                  
    0      0.742389   0.802226  
    1      0.75423    0.801032                                  
    2      0.728872   0.800519                                  



[0.8005188]

### Mini Net

In [39]:
class EmbeddingNet(nn.Module):
    def __init__(self, n_users, n_movies, nh=10, p1=0.05, p2=0.5):
        super().__init__()
        (self.u, self.m) = [get_emb(*o) for o in [
            (n_users, n_factors), (n_movies, n_factors)]]
        self.lin1 = nn.Linear(n_factors*2, nh)
        self.lin2 = nn.Linear(nh, 1)
        self.drop1 = nn.Dropout(p1)
        self.drop2 = nn.Dropout(p2)
        
    def forward(self, cats, conts):
        users,movies = cats[:,0],cats[:,1]
        x = self.drop1(torch.cat([self.u(users),self.m(movies)], dim=1))
        x = self.drop2(F.relu(self.lin1(x)))
        return F.sigmoid(self.lin2(x)) * (max_rating-min_rating+1) + min_rating-0.5

In [42]:
wd=1e-5
model = EmbeddingNet(n_users,n_movies).cuda()


In [43]:
opt = optim.Adam(model.parameters(), 1e-3, weight_decay=wd)

In [44]:
fit(model, data, 3, opt, F.mse_loss)

HBox(children=(IntProgress(value=0, description='Epoch', max=3), HTML(value='')))

epoch      trn_loss   val_loss                                  
    0      0.865774   0.805908  
    1      0.806391   0.791762                                  
    2      0.795513   0.783978                                  



[0.7839782]

In [45]:
set_lrs(opt, 1e-3)

In [46]:
fit(model, data, 3, opt, F.mse_loss)

HBox(children=(IntProgress(value=0, description='Epoch', max=3), HTML(value='')))

epoch      trn_loss   val_loss                                  
    0      0.76933    0.785682  
    1      0.766322   0.78825                                   
    2      0.757464   0.795563                                  



[0.7955631]